In [68]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm

from math import ceil
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import SelectPercentile as SP
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from tqdm import tqdm

In [377]:
df = pd.read_csv('train.csv', index_col = 'id')

X_test = pd.read_csv('test.csv', index_col='id')

# Dropping the rows which have missing prediction values
df = df.dropna(axis=0, subset=['price_doc'])

y_train = df.price_doc
X_train = df.drop(['price_doc'], axis = 1)

In [70]:
X_test.green_part_2000

id
30474    15.17
30475    49.26
30476    40.90
30477    31.35
30478    38.61
         ...  
38131    34.77
38132    18.54
38133     6.05
38134    10.24
38135     8.66
Name: green_part_2000, Length: 7662, dtype: float64

In [71]:
X_markets = X_train[['market_count_5000', 'market_count_3000', 'market_count_2000', 'market_count_1500', 'market_count_1000', 'market_count_500']]
X_markets.corr()

,market_count_5000,market_count_3000,market_count_2000,market_count_1500,market_count_1000,market_count_500
market_count_5000,1.000000,0.790549,0.581023,0.473395,0.363342,0.256790
market_count_3000,0.790549,1.000000,0.768959,0.625851,0.470855,0.314709
market_count_2000,0.581023,0.768959,1.000000,0.855601,0.665947,0.411397
market_count_1500,0.473395,0.625851,0.855601,1.000000,0.773842,0.479010
market_count_1000,0.363342,0.470855,0.665947,0.773842,1.000000,0.606724
market_count_500,0.256790,0.314709,0.411397,0.479010,0.606724,1.000000


In [72]:
df['market_count_5000'] = df['market_count_5000'] - df['market_count_3000']
df['market_count_3000'] = df['market_count_3000'] - df['market_count_2000']
df['market_count_2000'] = df['market_count_2000'] - df['market_count_1500']
df['market_count_1500'] = df['market_count_1500'] - df['market_count_1000']
df['market_count_1000'] = df['market_count_1000'] - df['market_count_500']

In [73]:
X_markets = X_train[['market_count_5000', 'market_count_3000', 'market_count_2000', 'market_count_1500', 'market_count_1000', 'market_count_500']]
X_markets.corr()

,market_count_5000,market_count_3000,market_count_2000,market_count_1500,market_count_1000,market_count_500
market_count_5000,1.000000,0.406967,0.223655,0.196833,0.152078,0.159390
market_count_3000,0.406967,1.000000,0.206479,0.124705,0.038776,0.083889
market_count_2000,0.223655,0.206479,1.000000,0.101866,0.098854,0.071359
market_count_1500,0.196833,0.124705,0.101866,1.000000,0.145249,0.127205
market_count_1000,0.152078,0.038776,0.098854,0.145249,1.000000,0.074969
market_count_500,0.159390,0.083889,0.071359,0.127205,0.074969,1.000000


In [74]:
df['sport_count_5000'] = df['sport_count_5000'] - df['sport_count_3000']
df['sport_count_3000'] = df['sport_count_3000'] - df['sport_count_2000']
df['sport_count_2000'] = df['sport_count_2000'] - df['sport_count_1500']
df['sport_count_1500'] = df['sport_count_1500'] - df['sport_count_1000']
df['sport_count_1000'] = df['sport_count_1000'] - df['sport_count_500']

In [75]:
df['leisure_count_5000'] = df['leisure_count_5000'] - df['leisure_count_3000']
df['leisure_count_3000'] = df['leisure_count_3000'] - df['leisure_count_2000']
df['leisure_count_2000'] = df['leisure_count_2000'] - df['leisure_count_1500']
df['leisure_count_1500'] = df['leisure_count_1500'] - df['leisure_count_1000']
df['leisure_count_1000'] = df['leisure_count_1000'] - df['leisure_count_500']

In [193]:
df['church_count_5000'] = df['church_count_5000'] - df['church_count_3000']
df['church_count_3000'] = df['church_count_3000'] - df['church_count_2000']
df['church_count_2000'] = df['church_count_2000'] - df['church_count_1500']
df['church_count_1500'] = df['church_count_1500'] - df['church_count_1000']
df['church_count_1000'] = df['church_count_1000'] - df['church_count_500']

In [ ]:
df['small_church_count_5000'] = df['church_count_5000']-df['big_church_count_5000']
df['total_church_count_5000'] = 1.1*df['big_church_count_5000']+0.9*df['small_church_count_5000']

df['small_church_count_3000'] = df['church_count_3000']-df['big_church_count_3000']
df['total_church_count_3000'] = 1.1*df['big_church_count_3000']+0.9*df['small_church_count_3000']

df['small_church_count_2000'] = df['church_count_2000']-df['big_church_count_2000']
df['total_church_count_2000'] = 1.1*df['big_church_count_2000']+0.9*df['small_church_count_2000']

df['small_church_count_1500'] = df['church_count_1500']-df['big_church_count_1500']
df['total_church_count_1500'] = 1.1*df['big_church_count_1500']+0.9*df['small_church_count_1500']

df['small_church_count_1000'] = df['church_count_1000']-df['big_church_count_1000']
df['total_church_count_1000'] = 1.1*df['big_church_count_1000']+0.9*df['small_church_count_1000']

df['small_church_count_500'] = df['church_count_500']-df['big_church_count_500']
df['total_church_count_500'] = 1.1*df['big_church_count_500']+0.9*df['small_church_count_500']

In [ ]:
df['cafe_count_5000'] = df['cafe_count_5000'] - df['cafe_count_3000']
df['cafe_count_3000'] = df['cafe_count_3000'] - df['cafe_count_2000']
df['cafe_count_2000'] = df['cafe_count_2000'] - df['cafe_count_1500']
df['cafe_count_1500'] = df['cafe_count_1500'] - df['cafe_count_1000']
df['cafe_count_1000'] = df['cafe_count_1000'] - df['cafe_count_500']

In [241]:
df['green_part_5000'] = df['green_part_5000'] - df['green_part_3000']
df['green_part_3000'] = df['green_part_3000'] - df['green_part_2000']
df['green_part_2000'] = df['green_part_2000'] - df['green_part_1500']
df['green_part_1500'] = df['green_part_1500'] - df['green_part_1000']
df['green_part_1000'] = df['green_part_1000'] - df['green_part_500']

In [238]:
df['office_count_5000'] = df['office_count_5000'] - df['office_count_3000']
df['office_count_3000'] = df['office_count_3000'] - df['office_count_2000']
df['office_count_2000'] = df['office_count_2000'] - df['office_count_1500']
df['office_count_1500'] = df['office_count_1500'] - df['office_count_1000']
df['office_count_1000'] = df['office_count_1000'] - df['office_count_500']

In [252]:
df['office_sqm_5000'] = df['office_sqm_5000'] - df['office_sqm_3000']
df['office_sqm_3000'] = df['office_sqm_3000'] - df['office_sqm_2000']
df['office_sqm_2000'] = df['office_sqm_2000'] - df['office_sqm_1500']
df['office_sqm_1500'] = df['office_sqm_1500'] - df['office_sqm_1000']
df['office_sqm_1000'] = df['office_sqm_1000'] - df['office_sqm_500']

In [ ]:
df['office_avg_5000'] = df['office_sqm_5000']/df['office_count_5000']
df['office_avg_3000'] = df['office_sqm_3000']/df['office_count_3000']
df['office_avg_2000'] = df['office_sqm_2000']/df['office_count_2000']
df['office_avg_1500'] = df['office_sqm_1500']/df['office_count_1500']
df['office_avg_1000'] = df['office_sqm_1000']/df['office_count_1000']

In [ ]:
df['prom_part_5000'] = df['prom_part_5000'] - df['prom_part_3000']
df['prom_part_3000'] = df['prom_part_3000'] - df['prom_part_2000']
df['prom_part_2000'] = df['prom_part_2000'] - df['prom_part_1500']
df['prom_part_1500'] = df['prom_part_1500'] - df['prom_part_1000']
df['prom_part_1000'] = df['prom_part_1000'] - df['prom_part_500']

In [ ]:
df['trc_count_5000'] = df['trc_count_5000'] - df['trc_count_3000']
df['trc_count_3000'] = df['trc_count_3000'] - df['trc_count_2000']
df['trc_count_2000'] = df['trc_count_2000'] - df['trc_count_1500']
df['trc_count_1500'] = df['trc_count_1500'] - df['trc_count_1000']
df['trc_count_1000'] = df['trc_count_1000'] - df['trc_count_500']

In [ ]:
df['trc_sqm_5000'] = df['trc_sqm_5000'] - df['trc_sqm_3000']
df['trc_sqm_3000'] = df['trc_sqm_3000'] - df['trc_sqm_2000']
df['trc_sqm_2000'] = df['trc_sqm_2000'] - df['trc_sqm_1500']
df['trc_sqm_1500'] = df['trc_sqm_1500'] - df['trc_sqm_1000']
df['trc_sqm_1000'] = df['trc_sqm_1000'] - df['trc_sqm_500']

In [ ]:
df['trc_avg_5000'] = df['trc_sqm_5000']/df['trc_count_5000']
df['trc_avg_3000'] = df['trc_sqm_3000']/df['trc_count_3000']
df['trc_avg_2000'] = df['trc_sqm_2000']/df['trc_count_2000']
df['trc_avg_1500'] = df['trc_sqm_1500']/df['trc_count_1500']
df['trc_avg_1000'] = df['trc_sqm_1000']/df['trc_count_1000']

In [234]:
X_train.trc_sqm_500.corr(X_train.trc_count_500)

0.7528232995105633

In [244]:
X_train.green_part_1000.corr(y_train)

0.0009599164168327278

In [267]:
X_train.avg_office_size.corr(y_train)

nan

In [262]:
X_train['avg_office_size'] = X_train['office_sqm_5000']/X_train['office_count_5000']

In [211]:
X_train['total_church_count_5000'] = 1.1*X_train['big_church_count_5000']+0.9*X_train['small_church_count_5000']

In [210]:
X_train.total_church_count_5000.corr(y_train)

0.2698825655778925

In [170]:
X_train.indust_part.corr(y_train)

-0.08261410128650946

In [239]:
X_train.office_count_5000.corr(y_train)

0.2657949156567468

In [118]:
X_train.cafe_count_5000.corr(y_train)

0.2315455504501285

In [129]:
X_train.office_count_5000.corr(y_train)

0.21924864733837932

In [130]:
X_train.office_sqm_5000.corr(y_train)

0.26997677589841385

In [127]:
X_train['trc_sqm_5000'].head(1)

id
1    4036616
Name: trc_sqm_5000, dtype: int64

In [128]:
X_train['trc_count_5000'].head(1)

id
1    52
Name: trc_count_5000, dtype: int64

In [132]:
X_train.avg_office_size.corr(y_train)

-0.14479169259037197

In [171]:
X_train['environmental_condition_5000'] = X_train['green_zone_part'] - X_train['indust_part']

In [172]:
X_train.environmental_condition_5000.corr(y_train)

-0.026124357701811388

In [98]:
X_train.cafe_count_500_na_price.value_counts()

0     26270
1      2843
9       610
2       447
3       124
4       105
5        42
7        12
6         8
10        4
8         3
13        2
11        1
Name: cafe_count_500_na_price, dtype: int64

In [80]:
X_train[church_count] = X_train['big_church_count']

id
1         13
2         15
3         11
4          4
5        135
        ... 
30469     15
30470     98
30471      2
30472      6
30473      7
Name: big_church_count_5000, Length: 30471, dtype: int64

In [81]:
X_train.church_count_5000

id
1         18
2         22
3         16
4          2
5        115
        ... 
30469     14
30470    120
30471      8
30472     14
30473      9
Name: church_count_5000, Length: 30471, dtype: int64

In [228]:
X_train.cafe_count_5000_price_500.corr(X_train.cafe_count_5000)

0.9979626304210566

In [217]:
X_train.cafe_count_5000_price_high.

id
1         0
2         0
3         0
4         1
5        17
         ..
30469     0
30470    24
30471     0
30472     1
30473     0
Name: cafe_count_5000_price_high, Length: 30471, dtype: int64

In [218]:
X_train.cafe_count_5000_price_4000

id
1          4
2          3
3          3
4          2
5        108
        ... 
30469      5
30470    128
30471      1
30472      1
30473      2
Name: cafe_count_5000_price_4000, Length: 30471, dtype: int64

In [220]:
X_train.cafe_count_5000_price_2500

id
1          9
2         15
3         10
4         11
5        319
        ... 
30469     15
30470    313
30471      1
30472     22
30473      5
Name: cafe_count_5000_price_2500, Length: 30471, dtype: int64

In [268]:
columns_to_drop = ['cafe_sum_5000_min_price_avg', 'cafe_sum_5000_max_price_avg', 'cafe_sum_3000_min_price_avg', 'cafe_sum_3000_max_price_avg', 'cafe_sum_2000_min_price_avg', 'cafe_sum_2000_max_price_avg', 'cafe_sum_1500_min_price_avg', 'cafe_sum_1500_max_price_avg', 'cafe_sum_1000_min_price_avg', 'cafe_sum_1000_max_price_avg', 'cafe_sum_500_min_price_avg', 'cafe_sum_500_max_price_avg', 'cafe_count_5000_price_500', 'cafe_count_5000_price_1000','cafe_count_5000_price_2500', 'cafe_count_5000_price_4000', 'cafe_count_5000_price_high', 'cafe_count_3000_price_500', 'cafe_count_3000_price_1000','cafe_count_3000_price_2500', 'cafe_count_3000_price_4000', 'cafe_count_3000_price_high', 'cafe_count_2000_price_500', 'cafe_count_2000_price_1000','cafe_count_2000_price_2500', 'cafe_count_2000_price_4000', 'cafe_count_2000_price_high', 'cafe_count_1500_price_500', 'cafe_count_1500_price_1000','cafe_count_1500_price_2500', 'cafe_count_1500_price_4000', 'cafe_count_1500_price_high', 'cafe_count_1000_price_500', 'cafe_count_1000_price_1000','cafe_count_1000_price_2500', 'cafe_count_1000_price_4000', 'cafe_count_1000_price_high', 'cafe_count_500_price_500', 'cafe_count_500_price_500','cafe_count_500_price_2500', 'cafe_count_500_price_4000', 'cafe_count_500_price_high', 'mosque_count_5000', 'mosque_count_3000', 'mosque_count_2000', 'mosque_count_1500', 'mosque_count_1000', 'mosque_count_500']

In [269]:
X_train['activities_count_5000'] = 1.75*X_train['leisure_count_5000'] + X_train['sport_count_5000'] 
X_train['activities_count_3000'] = 1.75*X_train['leisure_count_3000'] + X_train['sport_count_3000']
X_train['activities_count_2000'] = 1.75*X_train['leisure_count_2000'] + X_train['sport_count_2000']
X_train['activities_count_1500'] = 1.75*X_train['leisure_count_1500'] + X_train['sport_count_1500']
X_train['activities_count_1000'] = 1.75*X_train['leisure_count_1000'] + X_train['sport_count_1000']
X_train['activities_count_500'] = 1.75*X_train['leisure_count_500'] + X_train['sport_count_500']

In [436]:
X_train['shopping_count_5000'] = X_train['market_count_5000'] + X_train['trc_count_5000']

In [437]:
X_train.shopping_count_5000.corr(y_train)

0.2825342142060361

In [438]:
X_train.market_count_5000.corr(y_train)

0.19402068877457698

In [439]:
X_train.trc_count_5000.corr(y_train)

0.2893709109303365

In [442]:
X_train.trc_count_5000.corr(X_train.shopping_count_5000)

0.9936919150949283